<a href="https://colab.research.google.com/github/FREDY129053/Practise_Temp/blob/main/Practise_Veg_Fru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fredy129053","key":"732172104435df3745774c6e74467316"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition

... resuming from 805306368 bytes (1325450922 bytes left) ...
100% 1.98G/1.98G [00:21<00:00, 99.3MB/s]
100% 1.98G/1.98G [00:21<00:00, 62.4MB/s]


In [ ]:
!unzip fruit-and-vegetable-image-recognition.zip

In [10]:
import cv2
import pathlib

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras import layers

In [11]:
train_dir = pathlib.Path("/content/train")
test_dir = pathlib.Path("/content/test")
validation_dir = pathlib.Path("/content/validation")

img_height, img_width = 224, 224

In [12]:
# Загрузка датасетов
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir)
validation_ds = tf.keras.utils.image_dataset_from_directory(validation_dir)

class_names = dict(zip(train_ds.class_names, range(len(train_ds.class_names))))
num_classes = len(class_names)

Found 3115 files belonging to 36 classes.
Found 359 files belonging to 36 classes.
Found 351 files belonging to 36 classes.


In [13]:
# Генератор для загрузки и преобразования картинок
train_generator = ImageDataGenerator(
    preprocessing_function = mobilenet_v2.preprocess_input,
    rotation_range = 32,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest"
)

In [14]:
train = train_generator.flow_from_directory(
    train_dir,
    target_size = (img_height,img_width),
    # изображение имеет 3 цветовых канала
    color_mode = "rgb",
    # создаем бинарные признаки меток класса
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 123
)

validation = train_generator.flow_from_directory(
    validation_dir,
    target_size = (img_height,img_width),
    # изображение имеет 3 цветовых канала
    color_mode = "rgb",
    # создаем бинарные признаки меток класса
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 123
)

Found 3115 images belonging to 36 classes.
Found 351 images belonging to 36 classes.


In [15]:
mobilenet_ = MobileNetV2(
    input_shape = (img_height,img_width,3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'avg'
)

9406464/9406464 [==============================] - 0s 0us/step


In [16]:
mobilenet_.trainable = False

# Слои
inputs = mobilenet_.input
x = Dense(128, activation = 'relu')(mobilenet_.output)
x = Dense(128, activation = 'relu')(x)
outputs = Dense(num_classes , activation = 'softmax')(x)

In [17]:
mobilenet = Model(inputs = inputs, outputs = outputs)

early_stopping = EarlyStopping(
	monitor='val_loss',
	mode='min',
	patience = 2,
	verbose=1,
	restore_best_weights=True,
)
checkpoint = ModelCheckpoint('***/fruit224mobile.h5',
                        	monitor = 'val_loss',
                        	mode = 'min',
                       	save_best_only = True)

callbacks = [early_stopping, checkpoint]

In [18]:
test = train_generator.flow_from_directory(
    test_dir,
    target_size = (img_height,img_width),
    # изображение имеет 3 цветовых канала
    color_mode = "rgb",
    # создаем бинарные признаки меток класса
    class_mode = "categorical",
    batch_size = 32,
    shuffle = True,
    seed = 123
)

mobilenet.compile(optimizer='adam', loss ='categorical_crossentropy',metrics = ['accuracy'])

history = mobilenet.fit(
    train, validation_data = validation,
    batch_size = 32,
    epochs = 25,
    callbacks = callbacks
)

(eval_loss, eval_accuracy) = mobilenet.evaluate(test)

Found 359 images belonging to 36 classes.
Epoch 1/25
13/98 [==>...........................] - ETA: 3:29 - loss: 3.3941 - accuracy: 0.1058

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


98/98 [==============================] - ETA: 0s - loss: 1.8490 - accuracy: 0.5018

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


98/98 [==============================] - 260s 3s/step - loss: 1.8490 - accuracy: 0.5018 - val_loss: 0.5972 - val_accuracy: 0.8063
Epoch 2/25
98/98 [==============================] - 263s 3s/step - loss: 0.7450 - accuracy: 0.7628 - val_loss: 0.3688 - val_accuracy: 0.8775
Epoch 3/25
98/98 [==============================] - 260s 3s/step - loss: 0.5952 - accuracy: 0.8019 - val_loss: 0.3971 - val_accuracy: 0.8490
Epoch 4/25
20/98 [=====>........................] - ETA: 3:09 - loss: 0.4887 - accuracy: 0.8313

KeyboardInterrupt: 